## Deep Dive into Sentiment Analysis of Twitter Data 

### Notebook 1:- Data Visualization and Cleaning

#### Importing Libraries 

In [1]:
# importing necessary libraries and packages


#Basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# NLP related libraries and packages
import nltk


#


In [3]:
# new libraries that needs to be imported 
import nltk


#### Data Loading

Following data has been downloaded from [Kaggle](https://www.kaggle.com/datasets/kazanova/sentiment140/data)

In [8]:
# import dataset from kaggle
dataset_tweets = pd.read_csv("./dataset/training.1600000.processed.noemoticon.csv",encoding = 'ISO-8859-1', header=None, names=['target', 'ids', 'date', 'flag','user', 'text'])

#Viewing 20 samples of the dataset_tweets
dataset_tweets.sample(20)b

,target,ids,date,flag,user,text
792233,0,2326215527,Thu Jun 25 06:43:22 PDT 2009,NO_QUERY,annalovesyou345,no more coffee.
1260835,4,1998453861,Mon Jun 01 18:34:28 PDT 2009,NO_QUERY,emilyboldrin,catching up on everything! feeling a little be...
622167,0,2229093899,Thu Jun 18 15:16:27 PDT 2009,NO_QUERY,WaDuRosario,@omelet_805 OMg you did it again! NOOOO!!! Ahh...
344602,0,2015703550,Wed Jun 03 05:19:29 PDT 2009,NO_QUERY,ROYISOK,"I have a friend I've never seen, he hides his ..."
178384,0,1965954306,Fri May 29 16:51:07 PDT 2009,NO_QUERY,BillieTheWriter,Am listening to Darren Hayes's Spin. Am talkin...
1495570,4,2069955920,Sun Jun 07 16:50:23 PDT 2009,NO_QUERY,beingmyself,@coolhands hey hey
1547310,4,2182705442,Mon Jun 15 13:31:35 PDT 2009,NO_QUERY,frankiestarr,Eating healthy is good! Just have to do it eve...
557571,0,2204547834,Wed Jun 17 02:30:16 PDT 2009,NO_QUERY,starjamgirl,"The Creepshow for Soundwave, but no 3EB I hop..."
522640,0,2192827933,Tue Jun 16 07:36:30 PDT 2009,NO_QUERY,ricardozedan,"Working on a new TIM solution, moving to Madal..."
484226,0,2180692535,Mon Jun 15 10:48:55 PDT 2009,NO_QUERY,SaraahPcd,@MissChriis about the dolls 'cause i can see t...


In [9]:
# viewing unique values in flag
dataset_tweets['flag'].unique()

array(['NO_QUERY'], dtype=object)

In [10]:
# checking for null values in the dataset_tweets
dataset_tweets.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [11]:
# get the complete meta data info about the dataset_tweets
dataset_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   ids     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


Data regarding the tweets currently have following 7 columns (target, ids, date, flag, user, text) with no null values present. In total we have 1600000 entries or different tweets observation in the dataset for the analysis.

#### Dropping Unnecessary columns/ Data Cleaning

Currently we only need tweets in text column and target column to train and analyze the dataset. Hence, we will be removing other columns from the dataset_tweets. 

In [14]:
# remove unnecessary columns
dataset_tweets.drop(labels =['ids', 'date', 'flag', 'user'], axis = 1, inplace = True)
dataset_tweets.head(5)

In [21]:
# Checking for unique values in target column
print(dataset_tweets.target.unique())
print(dataset_tweets.target.nunique())

[0 4]
2


We only have 2 unique values in target column that is 0-> Negative and 4-> Positive. We will replace 4 with 1 for ease of understanding and working with the dataset.

In [22]:
dataset_tweets['target'] = dataset_tweets['target'].replace(4,1)
print(dataset_tweets.target.unique())

[0 1]


In [29]:
# Viewing target == 0 i.e Negative Tweets
dataset_tweets[dataset_tweets['target']==0].text.sample(20)

332225                   We missed you @maggieofthemoon!!! 
260479    i should be studying for finals...but i'm just...
355981    @THEORACLE2 yea girl I did! Still don't have a...
517171    Workin it at #GNC till 4 THEN #American Eagle ...
585804                 @Itsmeraych but your company smellZ 
411081    @ryanscene is print screening me on webcam  an...
51496                  i never got to sleep  lets hang out 
180007                        @zerpes You should be here!  
537426    McFly+Jb don't believe :O McFlt +Pcd ? Don't b...
514171    ahhhh i need tickets to see limp bizkit in lon...
604051       @smolFry Better than Thang? A B C's got THANG 
505531                            @quotetheselines oh boo, 
666936                                   Working out hurts 
729438    Guess what I'm doing? Socialising of course. B...
280397                  Ugh. Doc wants my wisdom teeth out 
174425    I was expecting the sun to come out today. I g...
722905                       @KaylaO wow

In [28]:
# Similarly viewing target ==1 i.e Positive tweets
dataset_tweets[dataset_tweets['target']==1].text.sample(20)

1375935                  Jason is going to sing me to sleep 
1382605    otw to pim with @iwettt for a new haircut &amp...
1368381    cool.  watching scary movie with my family.hah...
1514377           @NobyMetcalf I'm going in! And thank you! 
1441995                    what a beautiful day for airsoft 
834214     @bekaaahx gooddd blesssedd the brokeenn roaaad...
1175037    @chriscornell WTF? Weird... I don't know what ...
887766                         Is loving the wolverine game 
1338687           Watching back repeat of Robin Hood series 
1114220    Hey @backstreetboys hi guys... PERU GUYS IS WA...
930511                                just arrived in dubai 
1358312    lazy day yay!  omg so sleepy! and those guys a...
1218273               @joe_wiley ei man is that your nissan 
909932               @savagetaylor next year will be sweeet 
1555292     Gonna take my doggie to get her staples removed 
1232553    Good morning lovlies...   How was the weekend?...
1031914    I hv this frn

In [31]:
# Count the occurance of Negative and Positive tweets in dataset_tweets
dataset_tweets.target.value_counts()

0    800000
1    800000
Name: target, dtype: int64